# Identification of block diagonalization of the 8-site Z2 LGT HVA generators

In [1]:
import os
import logging
import numpy as np
import jax
import jax.numpy as jnp
from fastdla import lie_closure
from fastdla.generators.z2lgt_hva import *
from fastdla.sparse_pauli_sum import SparsePauliSum
from fastdla.generators.spin_chain import translation, translation_eigenspace
from fastdla.linalg.eigenspace import get_eigenspace
from fastdla.linalg.block_diagonalization import sbd_fast

logging.basicConfig(level=logging.WARNING)
logging.getLogger('fastdla').setLevel(logging.INFO)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '.99'
jax.config.update('jax_enable_x64', True)

In [2]:
def clean_array(arr):
    result = np.where(np.isclose(arr.real, 0.), 0., arr.real).astype(arr.dtype)
    if arr.dtype == np.complex128:
        result += 1.j * np.where(np.isclose(arr.imag, 0.), 0., arr.imag)
    return result

def print_state(subspace):
    num_qubits = np.log2(subspace.shape[0]).astype(int)
    for col, vector in enumerate(subspace.T):
        indices = np.nonzero(vector)[0]
        bindices = (indices[:, None] >> np.arange(num_qubits)[None, ::-1]) % 2
        bstrs = [''.join(f'{b}' for b in brow) for brow in bindices]
        svec = ' + '.join(f'({coeff})|{bidx}>' for bidx, coeff in zip(bstrs, vector[indices]))
        print(col, svec)

In [3]:
num_fermions = 4
num_links = 2 * num_fermions
coupling_j = 1.2
mass = 0.5

## Reference: Full-shape Hamiltonian projected to the gauss sector

In [4]:
if False:
    generators = z2lgt_hva_generators(num_fermions)
    gauss_projector = z2lgt_gauss_projector([1, -1] * num_fermions)

    # generators are normalized
    h_gauge = generators[0] * np.sqrt(num_links)
    h_mass = mass * np.sqrt(num_fermions) * (generators[1] - generators[2])
    h_hop = 0.5 * coupling_j * np.sqrt(2 * num_fermions) * (generators[3] + generators[4])
    hamiltonian = h_gauge + h_mass + h_hop
    hproj = hamiltonian @ gauss_projector
    evals_orig, evecs_orig = map(clean_array, jnp.linalg.eigh(hproj.to_matrix(npmod=jnp)))

## Gauge-fixed Hamiltonian and generator terms in the link DOF basis

In [5]:
paulis = []
for ilink in range(num_links):
    paulis.append('I' * ilink + 'Z' + 'I' * (num_links - ilink - 1))

solved_gauge = SparsePauliSum(paulis, np.ones(num_links))

paulis = []
for ilink in range(num_links - 1):
    paulis.append('I' * ilink + 'ZZ' + 'I' * (num_links - ilink - 2))
paulis.append('Z' + 'I' * (num_links - 2) + 'Z')

solved_mass = SparsePauliSum(paulis, mass * np.ones(num_links))

paulis = ['XI' + 'I' * (num_links - 3) + 'I', 'XZ' + 'I' * (num_links - 3) + 'Z']
for ilink in range(num_links - 2):
    paulis.append('I' * ilink + 'IXI' + 'I' * (num_links - ilink - 3))
    paulis.append('I' * ilink + 'ZXZ' + 'I' * (num_links - ilink - 3))
paulis.append('I' + 'I' * (num_links - 3) + 'IX')
paulis.append('Z' + 'I' * (num_links - 3) + 'ZX')

solved_hop = SparsePauliSum(paulis, coupling_j * np.full(len(paulis), 0.5))

if False:
    hsolved = solved_gauge + solved_mass + solved_hop
    solved_gauge_mat = solved_gauge.to_matrix(npmod=jnp)
    solved_mass_mat = solved_mass.to_matrix(npmod=jnp)
    solved_hop_mat = solved_hop.to_matrix(npmod=jnp)
    hsolved_mat = hsolved.to_matrix(npmod=jnp)
    evals_s, evecs_s = map(clean_array, jnp.linalg.eigh(hsolved_mat))

    np.allclose(evals_s[evals_s != 0.], evals_orig[evals_orig != 0.])

In [6]:
solved_generators = SparsePauliSumArray([solved_gauge.normalize()])

paulis = []
for ilink in range(0, num_links - 1, 2):
    paulis.append('I' * ilink + 'ZZ' + 'I' * (num_links - ilink - 2))

solved_generators.append(SparsePauliSum(paulis, np.full(len(paulis), 1. / np.sqrt(len(paulis)))))

paulis = []
for ilink in range(1, num_links - 1, 2):
    paulis.append('I' * ilink + 'ZZ' + 'I' * (num_links - ilink - 2))
paulis.append('Z' + 'I' * (num_links - 2) + 'Z')

solved_generators.append(SparsePauliSum(paulis, np.full(len(paulis), 1. / np.sqrt(len(paulis)))))

paulis = ['XI' + 'I' * (num_links - 3) + 'I', 'XZ' + 'I' * (num_links - 3) + 'Z']
for ilink in range(1, num_links - 2, 2):
    paulis.append('I' * ilink + 'IXI' + 'I' * (num_links - ilink - 3))
    paulis.append('I' * ilink + 'ZXZ' + 'I' * (num_links - ilink - 3))

solved_generators.append(SparsePauliSum(paulis, np.full(len(paulis), 1. / np.sqrt(len(paulis)))))

paulis = []
for ilink in range(0, num_links - 2, 2):
    paulis.append('I' * ilink + 'IXI' + 'I' * (num_links - ilink - 3))
    paulis.append('I' * ilink + 'ZXZ' + 'I' * (num_links - ilink - 3))
paulis.append('I' + 'I' * (num_links - 3) + 'IX')
paulis.append('Z' + 'I' * (num_links - 3) + 'ZX')

solved_generators.append(SparsePauliSum(paulis, np.full(len(paulis), 1. / np.sqrt(len(paulis)))))

gsolved_mat = np.array(solved_generators.to_matrices(npmod=jnp))

## U(1) zero-charge and translation invariant subspace

In [7]:
lconfigs = (np.arange(2 ** num_links)[:, None] >> np.arange(num_links)[None, ::-1]) % 2
zvals = 1 - 2 * lconfigs
u1_charges = np.sum(zvals * np.roll(zvals, -1, axis=1) * np.array([-1, 1] * num_fermions)[None, :], axis=1)
u1_zeros = np.nonzero(u1_charges == 0)[0]
u1_subspace = np.array(jax.nn.one_hot(u1_zeros, 2 ** num_links).T, dtype=np.complex128)
subspace = clean_array(translation_eigenspace(0, u1_subspace, num_spins=8, shift=2))

### Alternative manual construction of the translation eigenspace

In [8]:
t2_op = translation(num_links, shift=2)
zero_indices = set(u1_zeros)
sequences = set()
for start in u1_zeros:
    sequence = [start]
    while True:
        mapped_nonzero = np.nonzero(np.squeeze(t2_op(u1_subspace[:, u1_zeros == sequence[-1]])))[0]
        if mapped_nonzero.shape[0] != 1:
            raise ValueError(f'Index {idx} maps to superposition')
        if (idx := mapped_nonzero[0]) == start:
            break
        if idx not in zero_indices:
            raise ValueError(f'Index {idx} leaked out')
        sequence.append(idx)
    sequences.add(frozenset(sequence))

subspace = np.zeros((2 ** num_links, len(sequences)), dtype=np.complex128)
for iseq, sequence in enumerate(sorted(sequences, key=lambda x: len(x))):
    for idx in sequence:
        subspace[idx, iseq] = 1. / np.sqrt(len(sequence))

## Projection of the Gauss-solved generators onto U(1) & T2 eigenspace

In [9]:
mat_proj = clean_array(np.einsum('ij,kil,lm->kjm', subspace.conjugate(), gsolved_mat, subspace))

## Further block diagonalization

In [10]:
transform, blocks = sbd_fast(mat_proj, hermitian=True, return_blocks=True)

## Eigenvectors of the 1-dimensional subspaces

In [19]:
for iv in range(4):
    print_state(clean_array(subspace @ transform[:, iv:iv + 1]))

0 ((-0.3535533905932738+0j))|01101111> + ((0.3535533905932738+0j))|01111011> + ((0.3535533905932738+0j))|10110111> + ((-0.3535533905932738+0j))|10111101> + ((-0.3535533905932738+0j))|11011011> + ((0.3535533905932738+0j))|11011110> + ((0.3535533905932738+0j))|11101101> + ((-0.3535533905932738+0j))|11110110>
0 ((-0.14774634740150833+0j))|00010111> + ((0.14774634740150827+0j))|00011101> + ((0.32120245458201724+0j))|00101110> + ((-0.32120245458201724+0j))|00111010> + ((0.14774634740150827+0j))|01000111> + ((-0.14774634740150833+0j))|01011100> + ((-0.14774634740150833+0j))|01110001> + ((0.14774634740150827+0j))|01110100> + ((0.32120245458201724+0j))|10001011> + ((-0.32120245458201724+0j))|10001110> + ((-0.32120245458201724+0j))|10100011> + ((0.32120245458201724+0j))|10111000> + ((-0.14774634740150833+0j))|11000101> + ((0.14774634740150827+0j))|11010001> + ((0.32120245458201724+0j))|11100010> + ((-0.32120245458201724+0j))|11101000>
0 ((-0.3212024545820173+0j))|00010111> + ((0.321202454582017

In [18]:
print_state(subspace[:, :1])

0 ((1+0j))|10101010>


(256, 40)

## Computation of Lie closure

In [ ]:
dla_basis = clean_array(lie_closure(blocks[-1]))

INFO:fastdla._lie_closure_impl.matrix_jax:Current Lie algebra dimension: 4


Basis size 844; 2000th/355746 commutator [b[62], b[47]]


INFO:fastdla._lie_closure_impl.matrix_jax:Found 1020 new ops in 2.06s
INFO:fastdla._lie_closure_impl.matrix_jax:Current Lie algebra dimension: 1024


Basis size 1087; 4000th/590241 commutator [b[88], b[84]]
Basis size 1261; 6000th/794430 commutator [b[109], b[5]]
Basis size 1289; 10000th/830116 commutator [b[140], b[130]]
Basis size 1293; 12000th/835278 commutator [b[154], b[65]]
Basis size 1293; 14000th/835278 commutator [b[166], b[139]]
Basis size 1293; 16000th/835278 commutator [b[178], b[69]]


INFO:fastdla._lie_closure_impl.matrix_jax:Found 271 new ops in 6.99s


In [94]:
dla_basis.shape

(1295, 36, 36)